In [1]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
from matplotlib import patches
#import mplhep as hep
import os
from tqdm import tqdm
import ipywidgets as widget
from samples import samples
from utils import readTreeNominal
import itertools

Welcome to JupyROOT 6.18/04


/cvmfs/cms.cern.ch/slc7_amd64_gcc820/lcg/root/6.18.04-nmpfii/lib/ROOT.py:649: UserWarning: "from ROOT import *" is not supported under IPython
  warnings.warn( '"from ROOT import *" is not supported under IPython' )


In [2]:
ROOT.gInterpreter.Declare("""
float compute_weight( float norm, float triggerXSF, float triggerSF, float pileupWeight, float pileupJetIDWeight, float lepIdSF, float isoSF, float L1NonPrefiringProb_CommonCalc, float MCWeight_MultiLepCalc, float tthfWeight, float btagDeepJetWeight, float btagDeepJet2DWeight_HTnj ){
    return norm * triggerXSF * triggerSF * pileupWeight * pileupJetIDWeight * lepIdSF * isoSF * L1NonPrefiringProb_CommonCalc * ( MCWeight_MultiLepCalc / abs( MCWeight_MultiLepCalc ) ) * tthfWeight * btagDeepJetWeight * btagDeepJet2DWeight_HTnj;
}
""")

True

In [3]:
ROOT.gInterpreter.Declare("""
float compute_weight_ttbar( float norm, float triggerXSF, float triggerSF, float pileupWeight, float pileupJetIDWeight, float topPtWeight13TeV, float lepIdSF, float isoSF, float L1NonPrefiringProb_CommonCalc, float MCWeight_MultiLepCalc, float tthfWeight, float btagDeepJetWeight, float btagDeepJet2DWeight_HTnj ){
    return norm * triggerXSF * triggerSF * pileupWeight * pileupJetIDWeight * topPtWeight13TeV * lepIdSF * isoSF * L1NonPrefiringProb_CommonCalc * ( MCWeight_MultiLepCalc / abs( MCWeight_MultiLepCalc ) ) * tthfWeight * btagDeepJetWeight * btagDeepJet2DWeight_HTnj;
}
""")

True

In [4]:
def extended_ABCD( A, B, C, X, Y ):
    D_yield = B * X * C**2 / ( A**2 * Y )
    D_err   = D_yield * np.sqrt( 1/B + 1/X + 1/Y + 4/C + 4/A )
    return D_yield, D_err

def ABCD( A, B, C ):
    D_yield = B * C / A
    D_err = D_yield * np.sqrt( 1/B + 1/C + 1/D_yield )
    return D_yield, D_err

In [5]:
rPath = "root://cmseos.fnal.gov//store/user/jmanagan/"
rDir = rPath + "BtoTW_Oct2023_fullRun2"
nTree = "Events_Nominal"
fDATA = [
    "SingleElecRun2016APVB",
    "SingleElecRun2016APVC",
    "SingleElecRun2016APVD",
    "SingleElecRun2016APVE",
    "SingleElecRun2016APVF",
    "SingleElecRun2016F",
    "SingleElecRun2016G",
    "SingleElecRun2016H",
    "SingleElecRun2017B",
    "SingleElecRun2017C",
    "SingleElecRun2017D",
    "SingleElecRun2017E",
    "SingleElecRun2017F",
    "SingleElecRun2018A",
    "SingleElecRun2018B",
    "SingleElecRun2018C",
    "SingleElecRun2018D",
    "SingleMuonRun2016APVB",
    "SingleMuonRun2016APVC",
    "SingleMuonRun2016APVD",
    "SingleMuonRun2016APVE",
    "SingleMuonRun2016APVF",
    "SingleMuonRun2016F",
    "SingleMuonRun2016G",
    "SingleMuonRun2016H",
    "SingleMuonRun2017B",
    "SingleMuonRun2017C",
    "SingleMuonRun2017D",
    "SingleMuonRun2017E",
    "SingleMuonRun2017F",
    "SingleMuonRun2018A",
    "SingleMuonRun2018B",
    "SingleMuonRun2018C",
    "SingleMuonRun2018D"
]
fSIG = [
    "Bprime_M1400_2016",
    "Bprime_M1400_2017",
    "Bprime_M1400_2018",
]
fBKG = {
    "EWK": [
        "DYMHT12002016APV",
        "DYMHT12002016",
        "DYMHT12002017",
        "DYMHT12002018",
        "DYMHT2002016APV",
        "DYMHT2002016",
        "DYMHT2002017",
        "DYMHT2002018",
        "DYMHT25002016APV",
        "DYMHT25002016",
        "DYMHT25002017",
        "DYMHT25002018",
        "DYMHT4002016APV",
        "DYMHT4002016",
        "DYMHT4002017",
        "DYMHT4002018",
        "DYMHT6002016APV",
        "DYMHT6002016",
        "DYMHT6002017",
        "DYMHT6002018",
        "DYMHT8002016APV",
        "DYMHT8002016",
        "DYMHT8002017",
        "DYMHT8002018",
        "WW2016APV",
        "WW2016",
        "WW2017",
        "WW2018",
        "WZ2016APV",
        "WZ2016",
        "WZ2017",
        "WZ2018",
        "ZZ2016APV",
        "ZZ2016",
        "ZZ2017",
        "ZZ2018",
    ],
    "WJETS": [
        "WJetsHT12002016APV",
        "WJetsHT12002016",
        "WJetsHT12002017",
        "WJetsHT12002018",
        "WJetsHT2002016APV",
        "WJetsHT2002016",
        "WJetsHT2002017",
        "WJetsHT2002018",
        "WJetsHT25002016APV",
        "WJetsHT25002016",
        "WJetsHT25002017",
        "WJetsHT25002018",
        "WJetsHT4002016APV",
        "WJetsHT4002016",
        "WJetsHT4002017",
        "WJetsHT4002018",
        "WJetsHT6002016APV",
        "WJetsHT6002016",
        "WJetsHT6002017",
        "WJetsHT6002018",
        "WJetsHT8002016APV",
        "WJetsHT8002016",
        "WJetsHT8002017",
        "WJetsHT8002018",
    ],
    "TTBAR": [
        "TTMT10002016APV",
        "TTMT10002016",
        "TTMT10002017",
        "TTMT10002018",
        "TTMT7002016APV",
        "TTMT7002016",
        "TTMT7002017",
        "TTMT7002018",
        "TTTo2L2Nu2016APV0",
        "TTTo2L2Nu20160",
        "TTTo2L2Nu20170",
        "TTTo2L2Nu20180",
        "TTTo2L2Nu2016APV700",
        "TTTo2L2Nu2016700",
        "TTTo2L2Nu2017700",
        "TTTo2L2Nu2018700",
        "TTTo2L2Nu2016APV1000",
        "TTTo2L2Nu20161000",
        "TTTo2L2Nu20171000",
        "TTTo2L2Nu20181000",
        "TTToHadronic2016APV0",
        "TTToHadronic20160",
        "TTToHadronic20170",
        "TTToHadronic20180",
        "TTToHadronic2016APV700",
        "TTToHadronic2016700",
        "TTToHadronic2017700",
        "TTToHadronic2018700",
        "TTToHadronic2016APV1000",
        "TTToHadronic20161000",
        "TTToHadronic20171000",
        "TTToHadronic20181000",
        "TTToSemiLeptonic2016APV0",
        "TTToSemiLeptonic20160",
        "TTToSemiLeptonic20170",
        "TTToSemiLeptonic20180",
        "TTToSemiLeptonic2016APV700",
        "TTToSemiLeptonic2016700",
        "TTToSemiLeptonic2017700",
        "TTToSemiLeptonic2018700",
        "TTToSemiLeptonic2016APV1000",
        "TTToSemiLeptonic20161000",
        "TTToSemiLeptonic20171000",
        "TTToSemiLeptonic20181000",
    ],
    "ST": [
        "STs2016APV",
        "STs2016",
        "STs2017",
        "STs2018",
        "STt2016APV",
        "STt2016",
        "STt2017",
        "STt2018",
        "STtb2016APV",
        "STtb2016",
        "STtb2017",
        "STtb2018",
        "STtW2016APV",
        "STtW2016",
        "STtW2017",
        "STtW2018",
        "STtWb2016APV",
        "STtWb2016",
        "STtWb2017",
        "STtWb2018",
    ],
    "TTBARX": [
        "TTHB2016APV",
        "TTHB2016",
        "TTHB2017",
        "TTHB2018",
        "TTHnonB2016APV",
        "TTHnonB2016",
        "TTHnonB2017",
        "TTHnonB2018",
        "TTWl2016APV",
        "TTWl2016",
        "TTWl2017",
        "TTWl2018",
        "TTWq2016APV",
        "TTWq2016",
        "TTWq2017",
        "TTWq2018",
        "TTZM102016APV",
        "TTZM102016",
        "TTZM102017",
    "TTZM102018"
    ],
    "QCD": [
        "QCDHT10002016APV",
        "QCDHT10002016",
        "QCDHT10002017",
        "QCDHT10002018",
        "QCDHT15002016APV",
        "QCDHT15002016",
        "QCDHT15002017",
        "QCDHT15002018",
        "QCDHT20002016APV",
        "QCDHT20002016",
        "QCDHT20002017",
        "QCDHT20002018",
        "QCDHT2002016APV",
        "QCDHT2002016",
        "QCDHT2002017",
        "QCDHT2002018",
        "QCDHT3002016APV",
        "QCDHT3002016",
        "QCDHT3002017",
        "QCDHT3002018",
        "QCDHT5002016APV",
        "QCDHT5002016",
        "QCDHT5002017",
        "QCDHT5002018",
        "QCDHT7002016APV",
        "QCDHT7002016",
        "QCDHT7002017",
        "QCDHT7002018",
    ],   
}

In [6]:
fBase = "W_MT <= 200"
variables = [
    "NJets_DeepFlavL",
    "NJets_forward",
    "Bprime_mass",
    "gcJet_ST"
]
rDF = { "DATA": {}, "SIG": {}, "BKG": {} }
print( "Loading data:" )
for f in tqdm( fDATA ):
    samplename = samples[f].samplename.split('/')[1]
    year = (f.split('Run')[1])[:-1]
    #print(f, samplename, year)
    fChain = readTreeNominal( samplename, year, rPath)
    #rDF["DATA"][f] = ROOT.RDataFrame( fChain ).Filter( fBase ).AsNumpy( columns = variables )
print( "Loading signal:" )
# for f in tqdm( fSIG ):
#     samplename = samples[f].samplename.split('/')[1]
#     year = (f.split('Run')[1])[:-1]
#     print(f, samplename, year)
#     rDF_ = ROOT.RDataFrame( nTree, rDir + f ).Filter( fBase ).Define( "weight", "compute_weight( {}, triggerXSF, triggerSF, pileupWeight, pileupJetIDWeight, lepIdSF, isoSF, L1NonPrefiringProb_CommonCalc, MCWeight_MultiLepCalc, tthfWeight, btagDeepJetWeight, btagDeepJet2DWeight_HTnj )".format( xsec.norm[f] ) )
#     rDF["SIG"][f] = rDF_.AsNumpy( columns = variables + [ "weight" ] )
# for g in fBKG:
#     rDF["BKG"][g] = {}
#     print( "Loading background group: " + g )
#     for f in tqdm( fBKG[g] ):
#         if g == "TTBAR":
#             rDF_ = ROOT.RDataFrame( nTree, rDir + f ).Filter( fBase ).Define( "weight", "compute_weight_ttbar( {}, triggerXSF, triggerSF, pileupWeight, pileupJetIDWeight, topPtWeight13TeV, lepIdSF, isoSF, L1NonPrefiringProb_CommonCalc, MCWeight_MultiLepCalc, tthfWeight, btagDeepJetWeight, btagDeepJet2DWeight_HTnj )".format( xsec.norm[f] ) )
#         else:
#             rDF_ = ROOT.RDataFrame( nTree, rDir + f ).Filter( fBase ).Define( "weight", "compute_weight( {}, triggerXSF, triggerSF, pileupWeight, pileupJetIDWeight, lepIdSF, isoSF, L1NonPrefiringProb_CommonCalc, MCWeight_MultiLepCalc, tthfWeight, btagDeepJetWeight, btagDeepJet2DWeight_HTnj )".format( xsec.norm[f] ) )
#         rDF["BKG"][g][f] = rDF_.AsNumpy( columns = variables + [ "weight" ] )

Loading data:


100%|██████████| 34/34 [00:00<00:00, 18734.41it/s]

Loading signal:


## Get the yields in each region

In [11]:
def rel( limits, val, df ):
    if val >= limits[-1]: return df >= val
    else: return df == val
    return df == val

In [12]:
nJ = np.linspace( 4, 5, 2 )
nB = np.linspace( 0, 2, 3 )

yields = { "DATA": {}, "SIG": {}, "BKG": {} }
mc_count = { "BKG": {} }

for nJ_ in nJ:
    yields[ "DATA" ][nJ_] = {}
    yields[ "SIG" ][nJ_] = {}
    yields[ "BKG" ][nJ_] = {}
    mc_count[ "BKG" ][nJ_] = {}
    for nB_ in nB:
        yields[ "DATA" ][nJ_][nB_] = {}
        yields[ "SIG" ][nJ_][nB_] = {}
        yields[ "BKG" ][nJ_][nB_] = {}
        mc_count[ "BKG" ][nJ_][nB_] = 0
        for f in rDF[ "DATA" ]:
            mask = ( rel(  nJ, nJ_, rDF[ "DATA" ][f]["NJets_JetSubCalc"]  ) ) & ( rel( nB, nB_, rDF[ "DATA" ][f]["NJetsCSV_JetSubCalc"] ) )
            yields[ "DATA" ][nJ_][nB_][f] = np.sum( mask )
        for f in rDF[ "SIG" ]:
            mask = ( rel(  nJ, nJ_, rDF[ "SIG" ][f]["NJets_JetSubCalc"]  ) ) & ( rel( nB, nB_, rDF[ "SIG" ][f]["NJetsCSV_JetSubCalc"] ) )
            yields[ "SIG" ][nJ_][nB_][f] = np.sum( rDF[ "SIG" ][f][ "weight" ][mask] )
        for g in rDF[ "BKG" ]:
            yields[ "BKG" ][nJ_][nB_][g] = {}
            for f in rDF["BKG"][g]:
                mask = ( rel(  nJ, nJ_, rDF[ "BKG" ][g][f]["NJets_JetSubCalc"]  ) ) & ( rel( nB, nB_, rDF[ "BKG" ][g][f]["NJetsCSV_JetSubCalc"] ) )
                yields[ "BKG" ][nJ_][nB_][g][f] = np.sum( rDF[ "BKG" ][g][f][ "weight" ][ mask ] )
                if g == "TTBAR":
                    mc_count[ "BKG" ][nJ_][nB_] += int( np.sum(mask) )

Compute the group yields

In [13]:
for nJ_ in nJ:
    for nB_ in nB:
        print( "Yields for (NJ,NB) = ({},{})".format(nJ_,nB_) )
        yields[ "DATA" ][nJ_][nB_][ "TOTAL" ] = 0
        for f in rDF[ "DATA" ]:
            yields[ "DATA" ][nJ_][nB_][ "TOTAL" ] += yields[ "DATA" ][nJ_][nB_][f]
        print( "  - Data: {}".format( yields[ "DATA" ][nJ_][nB_][ "TOTAL" ] ) )
        yields[ "SIG" ][nJ_][nB_][ "TOTAL" ] = 0
        for f in rDF[ "SIG" ]:
            yields[ "SIG" ][nJ_][nB_][ "TOTAL" ] += yields[ "SIG" ][nJ_][nB_][f]
        print( "  - Signal: {:.2f}".format( yields[ "SIG" ][nJ_][nB_][ "TOTAL" ] ) )
        yields[ "BKG" ][nJ_][nB_][ "TOTAL" ] = 0
        for g in rDF[ "BKG" ]:
            yields[ "BKG" ][nJ_][nB_][g][ "TOTAL" ] = 0
            for f in rDF[ "BKG" ][g]:
                yields[ "BKG" ][nJ_][nB_][g][ "TOTAL" ] += yields[ "BKG" ][nJ_][nB_][g][f]
                yields[ "BKG" ][nJ_][nB_][ "TOTAL" ] += yields[ "BKG" ][nJ_][nB_][g][f]
            print( "  - Background ({}): {:.2f}".format( g, yields[ "BKG" ][nJ_][nB_][g][ "TOTAL" ] ) )
        print( "  - Total Background: {:.2f}".format( yields[ "BKG" ][nJ_][nB_][ "TOTAL" ] ) )

Yields for (NJ,NB) = (4.0,0.0)
  - Data: 379716
  - Signal: 0.22
  - Background (TTBAR): 64990.98
  - Background (TOP): 6025.12
  - Background (TTH): 81.80
  - Background (EWK): 237727.24
  - Background (QCD): 97648.80
  - Total Background: 406473.94
Yields for (NJ,NB) = (4.0,1.0)
  - Data: 307929
  - Signal: 0.85
  - Background (TTBAR): 232048.34
  - Background (TOP): 20110.78
  - Background (TTH): 298.34
  - Background (EWK): 45964.73
  - Background (QCD): 38907.84
  - Total Background: 337330.02
Yields for (NJ,NB) = (4.0,2.0)
  - Data: 239622
  - Signal: 1.35
  - Background (TTBAR): 222612.68
  - Background (TOP): 15050.24
  - Background (TTH): 545.76
  - Background (EWK): 7287.40
  - Background (QCD): 8287.17
  - Total Background: 253783.25
Yields for (NJ,NB) = (5.0,0.0)
  - Data: 154165
  - Signal: 0.39
  - Background (TTBAR): 41523.27
  - Background (TOP): 2785.18
  - Background (TTH): 98.55
  - Background (EWK): 76521.98
  - Background (QCD): 38991.36
  - Total Background: 15992

Get the relative composition of the background

In [14]:
breakdown = {}
for nJ_ in nJ:
    breakdown[nJ_] = {}
    for nB_ in nB:
        breakdown[nJ_][nB_] = {}
        print( "Background breakdown for (NJ,NB) = ({},{})".format(nJ_,nB_) )
        for g in rDF[ "BKG" ]:
            breakdown[nJ_][nB_][g] = yields[ "BKG" ][nJ_][nB_][g]["TOTAL"] / yields[ "BKG" ][nJ_][nB_]["TOTAL"]
            print( "  - {:<6}: {:.2f} %".format( g, breakdown[nJ_][nB_][g] * 100. ) )



Background breakdown for (NJ,NB) = (4.0,0.0)
  - TTBAR : 15.99 %
  - TOP   : 1.48 %
  - TTH   : 0.02 %
  - EWK   : 58.49 %
  - QCD   : 24.02 %
Background breakdown for (NJ,NB) = (4.0,1.0)
  - TTBAR : 68.79 %
  - TOP   : 5.96 %
  - TTH   : 0.09 %
  - EWK   : 13.63 %
  - QCD   : 11.53 %
Background breakdown for (NJ,NB) = (4.0,2.0)
  - TTBAR : 87.72 %
  - TOP   : 5.93 %
  - TTH   : 0.22 %
  - EWK   : 2.87 %
  - QCD   : 3.27 %
Background breakdown for (NJ,NB) = (5.0,0.0)
  - TTBAR : 25.96 %
  - TOP   : 1.74 %
  - TTH   : 0.06 %
  - EWK   : 47.85 %
  - QCD   : 24.38 %
Background breakdown for (NJ,NB) = (5.0,1.0)
  - TTBAR : 76.61 %
  - TOP   : 4.73 %
  - TTH   : 0.19 %
  - EWK   : 8.83 %
  - QCD   : 9.63 %
Background breakdown for (NJ,NB) = (5.0,2.0)
  - TTBAR : 90.41 %
  - TOP   : 4.76 %
  - TTH   : 0.48 %
  - EWK   : 1.88 %
  - QCD   : 2.47 %


In [15]:
print( "| (NJ,NB) | {:<6} | {:<6} | {:<6} | {:<6} | {:<6} | {:<10} |".format( "TTBAR", "TOP", "TTH", "EWK", "QCD", "Total #" ) )
for nJ_, nB_ in itertools.product( nJ, nB ):
    sNJ = "{}".format( int(nJ_) ) if nJ_ == nJ[-1] else int(nJ_)
    sNB = "{}".format( int(nB_) ) if nB_ == nB[-1] else int(nB_)
    print( "| ({:<2},{:<2}) | {:<6.2f} | {:<6.2f} | {:<6.2f} | {:<6.2f} | {:<6.2f} | {:<10.2f} |".format( sNJ, sNB, 100*breakdown[nJ_][nB_]["TTBAR"], 100*breakdown[nJ_][nB_]["TOP"], 100*breakdown[nJ_][nB_]["TTH"], 100*breakdown[nJ_][nB_]["EWK"], 100*breakdown[ nJ_][nB_]["QCD"], yields[ "BKG" ][nJ_][nB_]["TOTAL"] ) )

| (NJ,NB) | TTBAR  | TOP    | TTH    | EWK    | QCD    | Total #    |
| (4 ,0 ) | 15.99  | 1.48   | 0.02   | 58.49  | 24.02  | 406473.94  |
| (4 ,1 ) | 68.79  | 5.96   | 0.09   | 13.63  | 11.53  | 337330.02  |
| (4 ,2 ) | 87.72  | 5.93   | 0.22   | 2.87   | 3.27   | 253783.25  |
| (5 ,0 ) | 25.96  | 1.74   | 0.06   | 47.85  | 24.38  | 159920.35  |
| (5 ,1 ) | 76.61  | 4.73   | 0.19   | 8.83   | 9.63   | 224360.57  |
| (5 ,2 ) | 90.41  | 4.76   | 0.48   | 1.88   | 2.47   | 240240.83  |


### Signal region prediction without background subtraction
Define yields

In [16]:
def get_region( nJ_, nJ, nB_, nB ):
    if len(nJ) == 3:
        if nJ_ == nJ[0] and nB_ == nB[0]: return "X"
        if nJ_ == nJ[0] and nB_ >= nB[1]: return "Y"
        if nJ_ == nJ[1] and nB_ == nB[0]: return "A"
        if nJ_ == nJ[1] and nB_ >= nB[1]: return "C"
        if nJ_ >= nJ[2] and nB_ == nB[0]: return "B"
        return "D"
    else:
        if nJ_ == nJ[0] and nB_ == nB[0]: return "X"
        if nJ_ == nJ[0] and nB_ == nB[1]: return "A"
        if nJ_ == nJ[0] and nB_ >= nB[2]: return "B"
        if nJ_ >= nJ[1] and nB_ == nB[0]: return "Y"
        if nJ_ >= nJ[1] and nB_ == nB[1]: return "C"
        return "D"

In [17]:
yields_region = {
    "MC": { region: 0 for region in [ "A", "B", "C", "D", "X", "Y" ] },
    "Data": { region: 0 for region in [ "A", "B", "C", "D", "X", "Y" ] }
}

for nJ_ in nJ:
    for nB_ in nB:
        yields_region[ "MC" ][ get_region( nJ_, nJ, nB_, nB ) ] = int(yields[ "BKG" ][nJ_][nB_][ "TOTAL" ])
        yields_region[ "Data" ][ get_region( nJ_, nJ, nB_, nB ) ] = int(yields[ "DATA" ][nJ_][nB_][ "TOTAL" ])

ABCD vs Extended ABCD Prediction

In [18]:
print( "NJ = {}".format(nJ ) )
print( "NB = {}".format(nB) )

mcPred, mcPredSyst = ABCD( yields_region[ "MC" ][ "A" ], yields_region[ "MC" ][ "B" ], yields_region[ "MC" ][ "C" ] )
mcPredExt, mcPredSystExt = extended_ABCD( yields_region[ "MC" ][ "A" ], yields_region[ "MC" ][ "B" ], yields_region[ "MC" ][ "C" ], yields_region[ "MC" ][ "X" ], yields_region[ "MC" ][ "Y" ] )

print( "ABCD Prediction (MC): {:.2f} pm {:.2f} (Stat) pm {:.2f} (Syst)".format( mcPred, np.sqrt(mcPred), mcPredSyst ) )
print( "Extended ABCD Prediction (MC): {:.2f} pm {:.2f} (Stat) pm {:.2f} (Syst)".format( mcPredExt, np.sqrt(mcPredExt), mcPredSystExt ) )
print( "MC SF: {:.2f} pm {:.2f} (Stat)".format( yields_region[ "MC" ][ "D" ], np.sqrt(mcPred) ) )

dataPred, dataPredSyst = ABCD( yields_region[ "Data" ][ "A" ], yields_region[ "Data" ][ "B" ], yields_region[ "MC" ][ "C" ] )
dataPredExt, dataPredSystExt = extended_ABCD( yields_region[ "Data" ][ "A" ], yields_region[ "Data" ][ "B" ], yields_region[ "Data" ][ "C" ], yields_region[ "Data" ][ "X" ], yields_region[ "Data" ][ "Y" ] )

print( "ABCD Prediction (Data): {:.2f} pm {:.2f} (Stat) pm {:.2f} (Syst)".format( dataPred, np.sqrt(dataPred), dataPredSyst ) )
print( "Extended ABCD Prediction (Data): {:.2f} pm {:.2f} (Stat) pm {:.2f} (Syst)".format( dataPredExt, np.sqrt(dataPredExt), dataPredSystExt ) )
print( "Data: {:.2f}".format( yields_region[ "Data" ][ "D" ] ) )

NJ = [4. 5.]
NB = [0. 1. 2.]
ABCD Prediction (MC): 168792.44 pm 410.84 (Stat) pm 638.78 (Syst)
Extended ABCD Prediction (MC): 285346.39 pm 534.18 (Stat) pm 1856.73 (Syst)
MC SF: 240240.00 pm 410.84 (Stat)
ABCD Prediction (Data): 174590.87 pm 417.84 (Stat) pm 661.56 (Syst)
Extended ABCD Prediction (Data): 242975.56 pm 492.93 (Stat) pm 1657.39 (Syst)
Data: 219917.00


## Signal region prediction with background subtraction
In each case, the minor SF background is added back directly

In [19]:
minor_bkg = yields[ "BKG" ][nJ[-1]][nB[-1]][ "TOP" ]["TOTAL"] + yields[ "BKG" ][nJ[-1]][nB[-1]][ "TTH" ]["TOTAL"] + yields[ "BKG" ][nJ[-1]][nB[-1]][ "EWK" ]["TOTAL"] 
print( "The minor background in Region D (NJ,NB)=({},{}) is: {:.1f}".format( int(nJ[-1]), int(nB[-1]), minor_bkg ) )

The minor background in Region D (NJ,NB)=(5,2) is: 17105.3


In [20]:
def subtraction_relative( data, bkg_tot, bkg_major ):
    return ( bkg_major / bkg_tot ) * data
    
def subtraction_absolute( data, bkg_minor ):
    return float( data - bkg_minor )

def p_diff( x, y ):
    return abs( 100. * ( x - y ) / x )

### Option 1: Relative subtraction before prediction
Remove a fraction of the data in CR beforemaking the prediction, and then add the minor SF background back to SR

In [21]:
print( "Option 1: Relative subtraction before prediction in NJ = {}, NB = {}".format( nJ, nB ) )
yields_subRel_region = {
    "Data": { region: 0 for region in [ "A", "B", "C", "D", "X", "Y" ] }
}

for nJ_ in nJ:
    for nB_ in nB:
        yields_subRel_region[ "Data" ][ get_region( nJ_, nJ, nB_, nB ) ] = subtraction_relative( 
            int(yields[ "DATA" ][nJ_][nB_][ "TOTAL" ]), 
            int(yields[ "BKG" ][nJ_][nB_][ "TOTAL" ]) + int(yields[ "SIG" ][nJ_][nB_][ "TOTAL" ]), 
            int(yields[ "BKG" ][nJ_][nB_][ "TTBAR" ]["TOTAL"]) + int(yields[ "BKG" ][nJ_][nB_][ "QCD" ]["TOTAL"])
        )

mcPred, mcPredSyst = ABCD( yields_subRel_region[ "Data" ][ "A" ], yields_subRel_region[ "Data" ][ "B" ], yields_subRel_region[ "Data" ][ "C" ] )
mcPredExt, mcPredSystExt = extended_ABCD( yields_subRel_region[ "Data" ][ "A" ], yields_subRel_region[ "Data" ][ "B" ], yields_subRel_region[ "Data" ][ "C" ],  yields_subRel_region[ "Data" ][ "X" ], yields_subRel_region[ "Data" ][ "Y" ] )
        
print( "  - ABCD Prediction w/ subtraction (Data): {:.2f} + {:.2f} = {:.2f} ({:.2f} %)".format( mcPred, minor_bkg, mcPred + minor_bkg, p_diff( yields_region[ "Data" ][ "D" ], mcPred + minor_bkg ) ) )
print( "    + Transfer Factor: {:.5f}".format( mcPred / mc_count[ "BKG" ][nJ[-1]][nB[-1]] ) )
print( "  - ABCD Prediction (Data): {:.2f} ({:.2f} %)".format( dataPred, p_diff( yields_region[ "Data" ][ "D" ], dataPred ) ) )
print( "    + Transfer Factor: {:.5f}".format( dataPred / mc_count[ "BKG" ][nJ[-1]][nB[-1]] ) )
print( "  - Extended ABCD Prediction w/ subtraction (Data): {:.2f} + {:.2f} = {:.2f} ({:.2f} %)".format( mcPredExt, minor_bkg, mcPredExt + minor_bkg, p_diff( yields_region[ "Data" ][ "D" ], mcPredExt + minor_bkg ) ) )
print( "    + Systematic: {:.4f} ({:.2f}/{:.2f})".format( mcPredSystExt / mcPredExt, mcPredSystExt, mcPredExt ) )
print( "    + Statistical: {:.4f} ({:.2f}/{:.2f})".format( np.sqrt(mcPredExt)/mcPredExt, np.sqrt(mcPredExt), mcPredExt ) )
print( "    + Closure: {:.4f} ({:.2f}/{})".format( abs( mcPredExt + minor_bkg - yields_region["Data"]["D"] ) /yields_region["Data"]["D"], mcPredExt + minor_bkg, yields_region["Data"]["D"] ) )
print( "    + Transfer Factor: {:.5f}".format( mcPredExt / mc_count[ "BKG" ][nJ[-1]][nB[-1]] ) )
print( "  - Extended ABCD Prediction (Data): {:.2f} ({:.2f} %)".format( dataPredExt, p_diff( yields_region[ "Data" ][ "D" ], dataPredExt ) ) )
print( "    + Transfer Factor: {:.5f}".format( dataPredExt / mc_count[ "BKG" ][nJ[-1]][nB[-1]] ) )
print( "  - MC SF Prediction: {:.2f} ({:.2f} %)".format( yields[ "BKG" ][nJ[-1]][nB[-1]][ "TOTAL" ], p_diff( yields_region[ "Data" ][ "D" ], yields[ "BKG" ][nJ[-1]][nB[-1]][ "TOTAL" ] ) ) )
print( "  - Data: {}".format( yields_region[ "Data" ][ "D" ] ) )

Option 1: Relative subtraction before prediction in NJ = [4. 5.], NB = [0. 1. 2.]
  - ABCD Prediction w/ subtraction (Data): 150199.62 + 17105.26 = 167304.88 (23.92 %)
    + Transfer Factor: 0.01649
  - ABCD Prediction (Data): 174590.87 (20.61 %)
    + Transfer Factor: 0.01917
  - Extended ABCD Prediction w/ subtraction (Data): 202557.33 + 17105.26 = 219662.59 (0.12 %)
    + Systematic: 0.0080 (1616.64/202557.33)
    + Statistical: 0.0022 (450.06/202557.33)
    + Closure: 0.0012 (219662.59/219917)
    + Transfer Factor: 0.02224
  - Extended ABCD Prediction (Data): 242975.56 (10.49 %)
    + Transfer Factor: 0.02668
  - MC SF Prediction: 240240.83 (9.24 %)
  - Data: 219917


### Option 2: Absolute subtraction before prediction 
Remove a fixed amount of the data in CR when making the prediction, and then add the minor SF background back to SR

In [22]:
print( "Option 2: Absolute subtraction before prediction in NJ = {}, NB = {}".format( nJ, nB ) )
yields_subAbs_region = {
    "Data": { region: 0 for region in [ "A", "B", "C", "D", "X", "Y" ] }
}

for nJ_ in nJ:
    for nB_ in nB:
        yields_subAbs_region[ "Data" ][ get_region( nJ_, nJ, nB_, nB ) ] = subtraction_absolute( 
            int(yields[ "DATA" ][nJ_][nB_][ "TOTAL" ]), 
            yields[ "BKG" ][nJ_][nB_][ "TOP" ]["TOTAL"] + yields[ "BKG" ][nJ_][nB_][ "EWK" ]["TOTAL"] + yields[ "BKG" ][nJ_][nB_][ "TTH" ]["TOTAL"] # + yields[ "BKG" ][nJ_][nB_][ "QCD" ]["TOTAL"]
        )

mcPred, mcPredSyst = ABCD( yields_subAbs_region[ "Data" ][ "A" ], yields_subAbs_region[ "Data" ][ "B" ], yields_subAbs_region[ "Data" ][ "C" ] )
mcPredExt, mcPredSystExt = extended_ABCD( yields_subAbs_region[ "Data" ][ "A" ], yields_subAbs_region[ "Data" ][ "B" ], yields_subAbs_region[ "Data" ][ "C" ],  yields_subAbs_region[ "Data" ][ "X" ], yields_subAbs_region[ "Data" ][ "Y" ] )
        
print( "  - ABCD Prediction w/ subtraction (Data): {:.2f} + {:.2f} = {:.2f} ({:.2f} %)".format( mcPred, minor_bkg, mcPred + minor_bkg, p_diff( yields_region[ "Data" ][ "D" ], mcPred + minor_bkg ) ) )
print( "    + Transfer Factor: {:.5f}".format( mcPred / mc_count[ "BKG" ][nJ[-1]][nB[-1]] ) )
print( "  - ABCD Prediction (Data): {:.2f} ({:.2f} %)".format( dataPred, p_diff( yields_region[ "Data" ][ "D" ], dataPred ) ) )
print( "    + Transfer Factor: {:.5f}".format( dataPred / mc_count[ "BKG" ][nJ[-1]][nB[-1]] ) )
print( "  - Extended ABCD Prediction w/ subtraction (Data): {:.2f} + {:.2f} = {:.2f} ({:.2f} %)".format( mcPredExt, minor_bkg, mcPredExt + minor_bkg, p_diff( yields_region[ "Data" ][ "D" ], mcPredExt + minor_bkg ) ) )
print( "    + Systematic: {:.4f} ({:.2f}/{:.2f})".format( mcPredSystExt / mcPredExt, mcPredSystExt, mcPredExt ) )
print( "    + Statistical: {:.4f} ({:.2f}/{:.2f})".format( np.sqrt(mcPredExt)/mcPredExt, np.sqrt(mcPredExt), mcPredExt ) )
print( "    + Closure: {:.4f} ({:.2f}/{})".format( abs( mcPredExt + minor_bkg - yields_region["Data"]["D"] ) /yields_region["Data"]["D"], mcPredExt + minor_bkg, yields_region["Data"]["D"] ) )
print( "    + Transfer Factor: {:.5f}".format( mcPredExt / mc_count[ "BKG" ][nJ[-1]][nB[-1]] ) )
print( "  - Extended ABCD Prediction (Data): {:.2f} ({:.2f} %)".format( dataPredExt, p_diff( yields_region[ "Data" ][ "D" ], dataPredExt ) ) )
print( "    + Transfer Factor: {:.5f}".format( dataPredExt / mc_count[ "BKG" ][nJ[-1]][nB[-1]] ) )
print( "  - MC SF Prediction: {:.2f} ({:.2f} %)".format( yields[ "BKG" ][nJ[-1]][nB[-1]][ "TOTAL" ], p_diff( yields_region[ "Data" ][ "D" ], yields[ "BKG" ][nJ[-1]][nB[-1]][ "TOTAL" ] ) ) )
print( "  - Data: {}".format( yields_region[ "Data" ][ "D" ] ) )

Option 2: Absolute subtraction before prediction in NJ = [4. 5.], NB = [0. 1. 2.]
  - ABCD Prediction w/ subtraction (Data): 149593.12 + 17105.26 = 166698.38 (24.20 %)
    + Transfer Factor: 0.01643
  - ABCD Prediction (Data): 174590.87 (20.61 %)
    + Transfer Factor: 0.01917
  - Extended ABCD Prediction w/ subtraction (Data): 187664.99 + 17105.26 = 204770.25 (6.89 %)
    + Systematic: 0.0081 (1523.45/187664.99)
    + Statistical: 0.0023 (433.20/187664.99)
    + Closure: 0.0689 (204770.25/219917)
    + Transfer Factor: 0.02061
  - Extended ABCD Prediction (Data): 242975.56 (10.49 %)
    + Transfer Factor: 0.02668
  - MC SF Prediction: 240240.83 (9.24 %)
  - Data: 219917


### Option 3: Relative subtraction after prediction
Remove a fraction of the data in SR after the prediction, and then add the minor SF background back to the SR

In [23]:
print( "Option 3: Absolute subtraction after prediction in NJ = {}, NB = {}".format( nJ, nB ) )
yields_subRel_region = {
    "Data": { region: 0 for region in [ "A", "B", "C", "D", "X", "Y" ] }
}

for nJ_ in nJ:
    for nB_ in nB:
        yields_subRel_region[ "Data" ][ get_region( nJ_, nJ, nB_, nB ) ] = int(yields[ "DATA" ][nJ_][nB_][ "TOTAL" ])

mcPred, mcPredSyst = ABCD( yields_subRel_region[ "Data" ][ "A" ], yields_subRel_region[ "Data" ][ "B" ], yields_subRel_region[ "Data" ][ "C" ] )
mcPredExt, mcPredSystExt = extended_ABCD( yields_subRel_region[ "Data" ][ "A" ], yields_subRel_region[ "Data" ][ "B" ], yields_subRel_region[ "Data" ][ "C" ],  yields_subRel_region[ "Data" ][ "X" ], yields_subRel_region[ "Data" ][ "Y" ] )

mcPredSub = subtraction_relative( mcPred, yields[ "BKG" ][nJ[-1]][nB[-1]][ "TTBAR" ]["TOTAL"] + yields[ "BKG" ][nJ[-1]][nB[-1]][ "QCD" ]["TOTAL"], yields[ "BKG" ][nJ[-1]][nB[-1]]["TOTAL"] )
mcPredExtSub = subtraction_relative( mcPredExt, yields[ "BKG" ][nJ[-1]][nB[-1]][ "TTBAR" ]["TOTAL"] + yields[ "BKG" ][nJ[-1]][nB[-1]][ "QCD" ]["TOTAL"], yields[ "BKG" ][nJ[-1]][nB[-1]]["TOTAL"] )

print( "  - ABCD Prediction w/ subtraction (Data): {:.2f} + {:.2f} = {:.2f} ({:.2f} %)".format( mcPredSub, minor_bkg, mcPredSub + minor_bkg, p_diff( yields_region[ "Data" ][ "D" ], mcPredSub + minor_bkg ) ) )
print( "    + Transfer Factor: {:.5f}".format( mcPred / mc_count[ "BKG" ][nJ[-1]][nB[-1]] ) )
print( "  - ABCD Prediction (Data): {:.2f} ({:.2f} %)".format( dataPred, p_diff( yields_region[ "Data" ][ "D" ], dataPred ) ) )
print( "    + Transfer Factor: {:.5f}".format( dataPred / mc_count[ "BKG" ][nJ[-1]][nB[-1]] ) )
print( "  - Extended ABCD Prediction w/ subtraction (Data): {:.2f} + {:.2f} = {:.2f} ({:.2f} %)".format( mcPredExtSub, minor_bkg, mcPredExtSub + minor_bkg, p_diff( yields_region[ "Data" ][ "D" ], mcPredExtSub + minor_bkg ) ) )
print( "    + Transfer Factor: {:.5f}".format( mcPredExt / mc_count[ "BKG" ][nJ[-1]][nB[-1]] ) )
print( "  - Extended ABCD Prediction (Data): {:.2f} ({:.2f} %)".format( dataPredExt, p_diff( yields_region[ "Data" ][ "D" ], dataPredExt ) ) )
print( "    + Transfer Factor: {:.5f}".format( dataPredExt / mc_count[ "BKG" ][nJ[-1]][nB[-1]] ) )
print( "  - MC SF Prediction: {:.2f} ({:.2f} %)".format( yields[ "BKG" ][nJ[-1]][nB[-1]][ "TOTAL" ], p_diff( yields_region[ "Data" ][ "D" ], yields[ "BKG" ][nJ[-1]][nB[-1]][ "TOTAL" ] ) ) )
print( "  - Data: {}".format( yields_region[ "Data" ][ "D" ] ) )

Option 3: Absolute subtraction after prediction in NJ = [4. 5.], NB = [0. 1. 2.]
  - ABCD Prediction w/ subtraction (Data): 165533.58 + 17105.26 = 182638.84 (16.95 %)
    + Transfer Factor: 0.01688
  - ABCD Prediction (Data): 174590.87 (20.61 %)
    + Transfer Factor: 0.01917
  - Extended ABCD Prediction w/ subtraction (Data): 261601.73 + 17105.26 = 278706.99 (26.73 %)
    + Transfer Factor: 0.02668
  - Extended ABCD Prediction (Data): 242975.56 (10.49 %)
    + Transfer Factor: 0.02668
  - MC SF Prediction: 240240.83 (9.24 %)
  - Data: 219917


## Extended ABCD Transfer Factor Scale Factor for Applying Cuts

Calculate the Extended ABCD transfer factor scale factor after applying a cut

In [ ]:
nJ = np.linspace(4,5,2)
nB = np.linspace(0,2,3)
yields = {}
yields_cut = {}
dnn_cut = 0.50
fRegion = fBase

for f in tqdm( fDATA + fBKG ):
    yields[f] = {}
    yields_cut[f] = {}
    for nj in nJ:
        yields[f][nj] = {}
        yields_cut[f][nj] = {}
        if nj == nJ[-1]:
            mask_nj = ( rDF[f]["NJets_JetSubCalc"] >= nj )
        else:
            mask_nj = ( rDF[f]["NJets_JetSubCalc"] == nj )
        for nb in nB:
            if nb == nB[-1]:
                mask_nb = mask_nj & ( rDF[f]["NJetsCSV_JetSubCalc"] >= nb )
            else:
                mask_nb = mask_nj & ( rDF[f]["NJetsCSV_JetSubCalc"] == nb )
            if f in fBKG:
                mask_cut = mask_nb & ( rDF[f]["DNN_1to40_3t_nJ5pnB2p"] > dnn_cut )
            else:
                mask_cut = mask_nb & ( rDF[f]["DNN_1to40_3t"] > dnn_cut )
            yields[f][nj][nb] = np.sum( mask_nb )
            yields_cut[f][nj][nb] = np.sum( mask_cut )
            

yields["DATA"] = {}
yields_cut["DATA"] = {}
yields["BKG"] = {}
yields_cut["BKG"] = {}
for nj in nJ:
    yields["DATA"][nj] = {}
    yields_cut["DATA"][nj] = {}
    yields["BKG"][nj] = {}
    yields_cut["BKG"][nj] = {}
    for nb in nB:
        yields["DATA"][nj][nb] = 0
        yields_cut["DATA"][nj][nb] = 0
        yields["BKG"][nj][nb] = 0
        yields_cut["BKG"][nj][nb] = 0
        for f in tqdm( fDATA ):
            yields["DATA"][nj][nb] += yields[f][nj][nb]
            yields_cut["DATA"][nj][nb] += yields_cut[f][nj][nb]
        for f in tqdm( fBKG ):
            yields["BKG"][nj][nb] += yields[f][nj][nb]
            yields_cut["BKG"][nj][nb] += yields_cut[f][nj][nb]

In [ ]:
X = [nJ[0],nB[0]]
Y = [nJ[1],nB[0]]
C = [nJ[1],nB[1]]
A = [nJ[0],nB[1]]
B = [nJ[0],nB[2]]
D = [nJ[1],nB[2]]
pD, pD_err = extended_ABCD(
    X = int(yields["DATA"][X[0]][X[1]]),
    Y = int(yields["DATA"][Y[0]][Y[1]]),
    C = int(yields["DATA"][C[0]][C[1]]),
    A = int(yields["DATA"][A[0]][A[1]]),
    B = int(yields["DATA"][B[0]][B[1]])
)
pD_cut, pD_cut_err = extended_ABCD(
    X = int(yields_cut["DATA"][X[0]][X[1]]),
    Y = int(yields_cut["DATA"][Y[0]][Y[1]]),
    C = int(yields_cut["DATA"][C[0]][C[1]]),
    A = int(yields_cut["DATA"][A[0]][A[1]]),
    B = int(yields_cut["DATA"][B[0]][B[1]])
)

In [ ]:
efficiency = {
    "DATA": {},
    "BKG": {}
}
for nj in nJ:
    efficiency["DATA"][nj] = {}
    efficiency["BKG"][nj]  = {}
    for nb in nB:
        efficiency["DATA"][nj][nb] = yields_cut["DATA"][nj][nb] / yields["DATA"][nj][nb]
        efficiency["BKG"][nj][nb]  = yields_cut["BKG"][nj][nb] / yields["BKG"][nj][nb]

In [ ]:
print( "Extended ABCD predictions with base selection:")
print( "  Predicted Yield: {:.2f}".format( pD ) )
print( "  Observered Yield: {:.2f}".format( yields["DATA"][D[0]][D[1]] ) )
print( "  Data / Extended ABCD = {:.2f}".format( yields["DATA"][D[0]][D[1]] / pD ) )
print( "  Percent Difference: {:.2f}%".format( 100. * ( pD - yields["DATA"][D[0]][D[1]] ) / yields["DATA"][D[0]][D[1]] ) )
print( "With ABCDnn DNN > {}:".format( dnn_cut ) )
print( "  Predicted Yield with cut: {:.2f}".format( pD_cut ) )
print( "  Observered Yield with cut: {:.2f}".format( yields_cut["DATA"][D[0]][D[1]] ) )
print( "  Data / Extended ABCD = {:.2f}".format( yields_cut["DATA"][D[0]][D[1]] / pD_cut ) )
print( "  Percent Difference with cut: {:.2f}%".format( 100. * ( pD - yields_cut["DATA"][D[0]][D[1]] ) / yields_cut["DATA"][D[0]][D[1]] ) )
print( "Region | Data                          | MC                            |")
for nj in nJ:
    for nb in nB:
        if [nj,nb] == X: region = "X"
        elif [nj,nb] == Y: region = "Y"
        elif [nj,nb] == A: region = "A"
        elif [nj,nb] == B: region = "B"
        elif [nj,nb] == C: region = "C"
        elif [nj,nb] == D: region = "D"
        print( "{}      | {:<8.0f} / {:<8.0f} = {:<4.3f}   | {:<8.0f} / {:<8.0f} = {:<4.3f}   |".format(
            region,
            yields_cut["DATA"][nj][nb], yields["DATA"][nj][nb], efficiency["DATA"][nj][nb],
            yields_cut["BKG"][nj][nb], yields["BKG"][nj][nb], efficiency["BKG"][nj][nb],
        ) )
print( "Transfer Factor: {:.0f} / {:.0f} = {:.5f}".format(
    pD, yields["BKG"][D[0]][D[1]], pD / yields["BKG"][D[0]][D[1]]
) )
print( "Transfer Factor w/ cut: {:.0f} / {:.0f} = {:.5f}".format(
    pD_cut, yields_cut["BKG"][D[0]][D[1]], pD_cut / yields_cut["BKG"][D[0]][D[1]]
) )
print( "Extended ABCD yield w/ cut after transfer factor: {:.0f} x {:.5f} = {:.0f}".format(
    yields_cut["BKG"][D[0]][D[1]], pD / yields["BKG"][D[0]][D[1]], yields_cut["BKG"][D[0]][D[1]] * pD / yields["BKG"][D[0]][D[1]]
))
print( "  --> Data / Extended ABCD = {:.2f}".format( yields_cut["DATA"][D[0]][D[1]] / ( yields_cut["BKG"][D[0]][D[1]] * pD / yields["BKG"][D[0]][D[1]] ) ) )
print( "  --> Percent Difference: {:.2f}%".format( 100. * ( ( yields_cut["BKG"][D[0]][D[1]] * pD / yields["BKG"][D[0]][D[1]] ) - yields_cut["DATA"][D[0]][D[1]] ) / yields_cut["DATA"][D[0]][D[1]] ) )